### Practice: Parameter Efficient Fine-Tuning
In this notebook, you're gonna fine-tune large language models within limited GPU memory.

In [1]:
%pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 28.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 M

In [2]:
model_name = 'Enoch/llama-7b-hf'

# loading Llama tokenizer ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# ... and the model itself
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads()     # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

### Prompt tuning: the story of a fox (2 pts)

![img](https://i.imgur.com/Ux3qQAu.png) (source: theodd1souts.fandom.com)

In [3]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(10):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))


Output: <s>A quick brown fox jumps over the lazy dog.
A quick


What a blatant lie! This particular fox assures you that it didn't in fact jump over the lazy dog. No, sir! The fox was just minding its own business. __Your task is to train the model to say truth: no dog was jumped over today.__

In [4]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
outputs = model(**batch)

next_word_logits = outputs.logits[:, :-1]
true_next_tokens = batch['input_ids'][:, 1:]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

print("Loss:", loss)

Loss: tensor(3.0725, device='cuda:0', grad_fn=<NllLossBackward0>)


In [5]:
!nvidia-smi

Wed Feb 14 09:56:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              36W /  70W |   5463MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Except, we can't train the entire model - that would be 28GB gradients in float32. Instead, let's run [prompt tuning](https://arxiv.org/abs/2104.08691).

![img](https://i.imgur.com/VwNNKnb.png)


In [6]:
ss1 = torch.randn(1, 3, 16)
ss2 = torch.randn(5, 10, 16)

In [7]:
ss1

tensor([[[-0.5926, -1.9606,  0.6362,  0.0907, -0.0293, -0.0355, -1.2586,
          -0.2591,  0.2095, -0.9316, -0.3573, -1.1104, -0.8031,  0.2587,
          -1.0838,  1.8278],
         [ 0.4714,  0.4025, -0.3307,  0.0258, -0.6788,  1.5427,  0.1013,
          -0.8552, -0.4076, -0.9360, -0.2423, -1.7375,  1.0209,  0.5488,
           0.5977, -0.4245],
         [-1.8908, -1.1617, -2.0316,  1.2541, -1.1169,  0.0880, -0.3152,
           0.3377, -2.6092, -3.7058, -0.7062, -2.7144, -0.4042,  0.3187,
          -0.1382,  0.4074]]])

In [8]:
ss2

tensor([[[-1.1234e+00, -5.1182e-01, -4.8923e-01, -5.9807e-01,  1.9695e+00,
           7.6537e-01,  2.8963e+00, -1.7037e+00, -2.9737e-01,  3.5624e-01,
           4.7984e-01, -4.8558e-01, -1.2676e-01,  4.5269e-02,  7.9477e-01,
          -1.1418e+00],
         [ 2.1729e-01,  1.6402e+00,  7.5093e-01,  9.0357e-01, -8.4085e-01,
           2.5446e-01, -8.7341e-01, -1.3427e+00,  4.4551e-01,  9.4235e-01,
          -7.6110e-01, -9.3504e-01, -2.5252e+00,  1.6115e-01, -3.5339e-01,
          -1.4541e+00],
         [ 2.2729e-01,  3.6525e-01,  1.2419e+00,  1.5275e+00, -2.4883e-01,
          -5.2116e-01, -4.9059e-01,  4.9044e-01, -2.3467e+00,  1.0812e-01,
           4.3873e-01,  4.1736e-01,  2.0390e+00,  3.9174e-01,  5.0167e-01,
           7.5698e-01],
         [ 2.7728e-01,  5.9571e-01, -1.3176e-01,  3.5254e-01,  8.7250e-01,
           1.5285e-01,  3.9323e-01,  2.6035e-01, -3.0316e-01, -5.6031e-01,
           9.7061e-01,  1.1250e+00, -5.7654e-01, -1.8065e+00, -4.4033e-01,
           7.4682e-01],
    

In [9]:
torch.cat([ss1.repeat(5, 1, 1), ss2], dim=1)[0]

tensor([[-5.9265e-01, -1.9606e+00,  6.3617e-01,  9.0657e-02, -2.9308e-02,
         -3.5495e-02, -1.2586e+00, -2.5909e-01,  2.0951e-01, -9.3162e-01,
         -3.5725e-01, -1.1104e+00, -8.0308e-01,  2.5873e-01, -1.0838e+00,
          1.8278e+00],
        [ 4.7141e-01,  4.0254e-01, -3.3068e-01,  2.5830e-02, -6.7879e-01,
          1.5427e+00,  1.0129e-01, -8.5525e-01, -4.0757e-01, -9.3599e-01,
         -2.4233e-01, -1.7375e+00,  1.0209e+00,  5.4883e-01,  5.9765e-01,
         -4.2449e-01],
        [-1.8908e+00, -1.1617e+00, -2.0316e+00,  1.2541e+00, -1.1169e+00,
          8.7993e-02, -3.1524e-01,  3.3772e-01, -2.6092e+00, -3.7058e+00,
         -7.0623e-01, -2.7144e+00, -4.0416e-01,  3.1871e-01, -1.3816e-01,
          4.0741e-01],
        [-1.1234e+00, -5.1182e-01, -4.8923e-01, -5.9807e-01,  1.9695e+00,
          7.6537e-01,  2.8963e+00, -1.7037e+00, -2.9737e-01,  3.5624e-01,
          4.7984e-01, -4.8558e-01, -1.2676e-01,  4.5269e-02,  7.9477e-01,
         -1.1418e+00],
        [ 2.1729e-01

In [10]:
ss2[0]

tensor([[-1.1234e+00, -5.1182e-01, -4.8923e-01, -5.9807e-01,  1.9695e+00,
          7.6537e-01,  2.8963e+00, -1.7037e+00, -2.9737e-01,  3.5624e-01,
          4.7984e-01, -4.8558e-01, -1.2676e-01,  4.5269e-02,  7.9477e-01,
         -1.1418e+00],
        [ 2.1729e-01,  1.6402e+00,  7.5093e-01,  9.0357e-01, -8.4085e-01,
          2.5446e-01, -8.7341e-01, -1.3427e+00,  4.4551e-01,  9.4235e-01,
         -7.6110e-01, -9.3504e-01, -2.5252e+00,  1.6115e-01, -3.5339e-01,
         -1.4541e+00],
        [ 2.2729e-01,  3.6525e-01,  1.2419e+00,  1.5275e+00, -2.4883e-01,
         -5.2116e-01, -4.9059e-01,  4.9044e-01, -2.3467e+00,  1.0812e-01,
          4.3873e-01,  4.1736e-01,  2.0390e+00,  3.9174e-01,  5.0167e-01,
          7.5698e-01],
        [ 2.7728e-01,  5.9571e-01, -1.3176e-01,  3.5254e-01,  8.7250e-01,
          1.5285e-01,  3.9323e-01,  2.6035e-01, -3.0316e-01, -5.6031e-01,
          9.7061e-01,  1.1250e+00, -5.7654e-01, -1.8065e+00, -4.4033e-01,
          7.4682e-01],
        [ 3.0520e-01

In [11]:
class WordEmbeddingsWithLearnedPrompts(nn.Module):
    """
    To perform prompt tuning, you will need to replace model's original word embeddings with a layer - THIS layer
     - that inserts trainable prompts instead of the first N token embeddings. """

    def __init__(self, word_embeddings: nn.Embedding, num_prompts: int):
        super().__init__()
        self.original_word_embeddings = word_embeddings
        self.num_prompts = num_prompts
        self.learnable_prompts = nn.Parameter(
            torch.randn(1, num_prompts, word_embeddings.embedding_dim), requires_grad=True)

    def forward(self, input_ids: torch.LongTensor):
        # input_ids shape: [batch_size, seq length]
        assert input_ids.dtype == torch.int64
        assert input_ids.shape[1] > self.num_prompts
        assert torch.all(input_ids[:, :self.num_prompts] == tokenizer.pad_token_id).item(), "don't forget to prepend several BOS tokens to input_ids"

        # Your task: embed input_ids, but replace the first :num_prompts: tokens with self.learnable_prompts
        # This is because we will prepend :num_prompts: padding tokens at the beginning

        # After you are done, you must produce a word embedding vector for each token in input_ids,
        # except that the first :num_prompts: vectors should equal learnable_prompts;
        # any additional vectors after first :num_prompts: ones should be embedded as usual
        # Note: since you're dealing with trainable params, please torch.cat instead of item assignment
        batch_size = input_ids.shape[0]

        input_text = input_ids[:, self.num_prompts:]
        embedded_input_text = self.original_word_embeddings(input_text)

        padded_input_text = torch.cat([self.learnable_prompts.repeat(batch_size, 1, 1), embedded_input_text], dim=1)

        return padded_input_text

In [12]:
num_prompts = 16
test_emb_layer = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)
test_input_ids = tokenizer("a cat say on a may", return_tensors='pt')['input_ids'].to(device)

space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                               dtype=torch.int64, device=device)
test_inputs_with_prompts = torch.cat([space_for_prompts, test_input_ids], dim=1)

with torch.cuda.amp.autocast():
  test_prompt_embeddings = test_emb_layer(test_inputs_with_prompts)

assert test_prompt_embeddings.shape[:2] == test_inputs_with_prompts.shape
assert test_prompt_embeddings.shape[-1] == model.config.hidden_size
assert torch.allclose(test_prompt_embeddings[:, :num_prompts], test_emb_layer.learnable_prompts.float())
assert torch.allclose(test_prompt_embeddings[:, num_prompts:], model.model.embed_tokens(test_input_ids).float())
print("Looks legit!")

Looks legit!


__Now that it works,__ let's inject learnable prompts into the main model and teach it about foxes.

In [13]:
assert isinstance(model.model.embed_tokens, nn.Embedding), "you have already replaced the embedding layer. If the replacement is broken, please reload the model"

model.model.embed_tokens = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)

opt = torch.optim.Adam([model.model.embed_tokens.learnable_prompts], lr=0.01)

In [14]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                               dtype=torch.int64, device=device)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)

outputs = model(**batch)
next_word_logits = outputs.logits[:, num_prompts : -1, :]
true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
print("Loss:", loss)

for i in range(150):
    outputs = model(**batch)
    next_word_logits = outputs.logits[:, num_prompts : -1, :]
    true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
    print("Loss:", loss)

    opt.zero_grad()
    loss.backward()

    opt.step()
# raise NotImplemented("Your task: iteratively train the model to reduce loss using prompt optimizer (opt)")


assert loss.item() <= 0.1
print("Good job!")

Loss: tensor(7.5856, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(7.5856, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(6.8700, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(6.3567, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.8588, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.3120, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.0244, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.5488, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.0203, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.7920, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.5939, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.3671, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.1052, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.8509, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(2.6461, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tens

In [15]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)


for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0, num_prompts:].cpu().numpy().tolist()))

# if you did everything right, the model will deny that the fox jumped over the lazy dog


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Using HuggingFace PEFT (2 points)

[`peft`](https://huggingface.co/docs/peft/index) is a transformer's sister library that allows you to apply various __p__arameter __e__fficient __f__ine-__t__uning methods to pre-trained transformers. The library imlements both prompt tuning, prefix tuning, as well as several adapter-based techniques under a common interface:



In [16]:
model_name = 'Enoch/llama-7b-hf'

# loading Llama tokenizer ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# ... and the model itself
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads()     # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [17]:
import peft
assert isinstance(model.model.embed_tokens, nn.Embedding), "please reload the model"

peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM, num_virtual_tokens=16)
model = peft.get_peft_model(model, peft_config)  # note: for most peft methods, this line also modifies model in-place
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))

Trainable parameters: 65536
Total parameters (excluding quantization): 3500478464


In [18]:
# Your task: optimize the PEFT-wrapped model to achieve next token prediction loss < 0.1, but this time using PEFT
# Please note: you no longer need to prepend PAD tokens, but you still need to skip :num_virtual_tokens: first logits.
# Finally, generate the sentence to make sure that the model learned the truth.
num_virtual_tokens = 16
opt = torch.optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.01)

the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)

outputs = model(**batch)
next_word_logits = outputs.logits[:, num_virtual_tokens:-1, :]
true_next_tokens = batch['input_ids'][:, 1:]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
print("Loss:", loss)

for i in range(150):
    outputs = model(**batch)
    next_word_logits = outputs.logits[:, num_virtual_tokens:-1, :]
    true_next_tokens = batch['input_ids'][:, 1:]
    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
    print("Loss:", loss)

    opt.zero_grad()
    loss.backward()

    opt.step()
# raise NotImplemented("Your task: iteratively train the model to reduce loss using prompt optimizer (opt)")


assert loss.item() <= 0.1
print("Good job!")

Loss: tensor(7.2685, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(7.2685, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(6.6902, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(6.2399, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.8085, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.4240, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(5.0883, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.7915, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.5327, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.3159, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(4.1148, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.9144, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.7208, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.5241, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tensor(3.3155, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss: tens

In [19]:
# Feel free to structure your code as you see fit - as long as it's legible :)
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))

# if you did everything right, the model will deny that the fox jumped over the lazy dog


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Parameter-efficient finetuning with LoRA (2 points)

When training on more serious tasks, you can use low-rank adapters based on the [LoRA paper](https://arxiv.org/pdf/2106.09685.pdf).

The core idea is to add low-rank adapters __in parallel with existing linear layers,__ like this:
<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>

In the original LoRA paper, the adapters were only added to attention projection matrices. However, [subsequent works](https://arxiv.org/abs/2305.14314) show that it is useful to adapt FFNs as well. But before we do any training, we need to implement the basic LoRA layer.

In [20]:
# re-load the model to remove any previous PEFT tuners
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [3]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module  # pre-trained (frozen) linear layer
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (self.module outputs + adapter outputs)
        #  <YOUR CODE HERE>
        module_outputs = self.module(input)
        adapter_outputs = (input @ self.adapter_A) @ self.adapter_B
        return module_outputs + adapter_outputs

In [22]:
# test your implementation
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

All tests passed!


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [23]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

In [24]:
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast(dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)
print("Grad check successful, well done!")

Grad check successful, well done!


### (example) How to train your model

The example below shows how to train the LoRA adapters on a dummy dataset. You will need to run a _similar_ training task later.

__Note:__ please scroll down for the homework task

In [25]:
# checking if the model can learn. Change max_steps for proper training
import datasets
data = datasets.load_dataset("Abirate/english_quotes", split="train[:32]") # 32 lines
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=1,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=250, max_steps=100, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

# NOTE: this is just an example! you do not have to wait for this progressbar to finish :)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.891200
2,1.696000
3,0.897000
4,1.745800
5,1.170100
6,0.734200
7,1.537800
8,1.091200
9,0.676800
10,1.447100


TrainOutput(global_step=100, training_loss=0.5403704797104001, metrics={'train_runtime': 160.3202, 'train_samples_per_second': 1.248, 'train_steps_per_second': 0.624, 'total_flos': 621258424123392.0, 'train_loss': 0.5403704797104001, 'epoch': 6.25})

### Final task: *actually* train the model (4 points)

Your task is to fine-tune the model to _generate python code_. Please use the above examples for inspiration. More specifically,

* __dataset:__ use [codeparrot-clean](https://huggingface.co/datasets/codeparrot/codeparrot-clean) or any other data containing python code. Since you do not need much data for this excercise, it is enough to use just shorter validation subset of `codeparrots`
* __preprocessing:__ select python code based on file extentions (.py)  (may skip in case of codeparrot - it is 100% python)
* __short lines:__ please take the first 512 characters of each line
* __adapter type:__ please use LoRA as defined above __plus at least one of:__
   - extra adapter on lm_head
   - extra adapter on MLP components (mlp.*)
   - trainable input embeddings (requires tweaking memory usage)

* __training:__ you do not have to train to convergence. If all goes well, your model should `.generate` code after 500 steps. Please use batch size of at least 4 (4 x 1 x 512 tokens) using `gradient_accumulation_steps=4`.


Note: the peft library also has LoRA implementation. However, we ask that for this assignment you show at least one complete training run with your own LoRA code.

__Alternative assignment:__ Instead of doing python code, feel free to substitute the task with any other dataset, e.g. your favorite artist or podcast, as long as it's ethical. If you choose your own task, please show examples of what your model learned - or did not learn, akin to the code examples below.

In [ ]:
# re-load the model to remove any previous PEFT tuners
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [4]:
import datasets
python_dataset = datasets.load_dataset("codeparrot/codeparrot-clean-valid")

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [32]:
python_dataset['train'][0]['content'][:512]

'from django.conf import settings\nfrom django.core.paginator import Paginator, InvalidPage, EmptyPage\nfrom django.http import HttpResponse\n\nfrom tardis.tardis_portal.auth import decorators as authz\nfrom tardis.tardis_portal.models import Dataset\nfrom tardis.tardis_portal.shortcuts import get_experiment_referer\nfrom tardis.tardis_portal.shortcuts import render_response_index\n\n\n@authz.dataset_access_required\ndef view_full_dataset(request, dataset_id):\n    """Displays a MX Dataset and associated information.\n\n '

In [37]:
trunc_data = python_dataset['train'].map(lambda samples: {'content': samples['content'][:512]})

In [45]:
len(trunc_data['content'][100])

512

In [46]:
data = trunc_data.map(lambda samples: tokenizer(samples['content']),
                                   batched=True)

Map:   0%|          | 0/61373 [00:00<?, ? examples/s]

In [65]:
prompts =  ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch']  # feel free to add a few more that are not 100% assiciated with Python

# <A WHOLE LOT OF YOUR CODE>
# generate baseline samples with the selected prompts before finetuning
# please feel free to use transformers.Trainer (as above) or your custom training code
# after the training concludes, please show examples of text generated by your model. It is expected to look like Python code fragments
# print the generation examples nicely (suggestion: use pandas or HTML) for easier comparison
# note: your LoRA-enhanced model can run generation the same way as the non-trained model (above)

for prompt in prompts:
    print('prompt:', prompt)
    print('*'*30)
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    print(tokenizer.decode(model.generate(**batch, max_new_tokens=25)[0]))
    print('='*50)

prompt: 
******************************
<s>▶▶ 2019-2020 School Year
The 2019-202
prompt: import
******************************
<s>import Foundation

public extension NSURL {
    public var absoluteString: String {
        return String(cString:
prompt: from
******************************
<s>from __future__ import absolute_import
from __future__ import division
from __future__ import print_
prompt: while
******************************
<s>while(1)
while(1) {
    // do something
}
\end{code}

Comment
prompt: try
******************************
<s>try to find the best solution for your needs.
We are a team of professionals with a long experience in the field of
prompt: if
******************************
<s>if ( !window.atmosphere ) {
    window.atmosphere = {};
}

(function ()
prompt: for
******************************
<s>for the 2019-2020 school year.
The application process for the 2019
prompt: torch
******************************
<s>torchbearer 2017-05-18 19:55:25 UTC #


In [66]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

In [67]:
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1, gradient_accumulation_steps=4,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=250, max_steps=500, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.505100
2,1.349000
3,1.522700
4,1.345500
5,0.870900
6,1.829900
7,1.253000
8,0.872600
9,1.327400
10,1.465500


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


UnboundLocalError: local variable 'active_adapters' referenced before assignment

In [69]:
model.eval()
with torch.no_grad():
  for prompt in prompts:
      print('prompt:', prompt)
      print('*'*30)
      batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
      print(tokenizer.decode(model.generate(**batch, max_new_tokens=25)[0]))
      print('='*50)

prompt: 
******************************
<s># -*- coding: utf-8 -*-
# Copyright 2015 Google Inc. All
prompt: import
******************************
<s>import os
import sys
import time
import logging
import logging.handlers
import logging.config
import logging.
prompt: from
******************************
<s>from django.db import models
from django.utils.translation import ugettext_lazy as _

from
prompt: while
******************************
<s>while True:
    try:
        print("Enter a number:")
        number = int(input())
       
prompt: try
******************************
<s>try:
    from urllib.parse import urlparse
except ImportError:
    from urlparse import urlparse

prompt: if
******************************
<s>if ( ! defined( 'ABSPATH' ) ) {
    exit;
}

/**
 * @package
prompt: for
******************************
<s>for (var i = 0; i < 100000000000000
prompt: torch
******************************
<s>torch.setdefaulttensortype('torch.FloatTensor')


def _get_batch_size


In [70]:
untrained_model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in untrained_model.parameters():
    param.requires_grad=False
untrained_model.gradient_checkpointing_enable()
untrained_model.enable_input_require_grads()

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [74]:
# This template helps to compare generated code samples in pretty table form
# feel free to present your work in other forms

from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

rows = []

with torch.no_grad():
  for prompt in prompts:
      batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

      before = tokenizer.decode(untrained_model.generate(**batch, max_new_tokens=25)[0]).replace('<s>', '')
      after = tokenizer.decode(model.generate(**batch, max_new_tokens=25)[0]).replace('<s>', '')
      # replace placeholders in the format() arguments
      rows.append(row_template.format(prompt, before, after))

display(HTML(table_template.format('\n'.join(rows))))

PROMPT,BEFORE,AFTER
``,▶▶ 2019-2020 School Year The 2019-202,# -*- coding: utf-8 -*- # Copyright 2015 Google Inc. All
`import`,import Foundation public extension NSURL { public var absoluteString: String { return String(cString:,import os import sys import time import logging import logging.handlers import logging.config import logging.
`from`,from __future__ import absolute_import from __future__ import division from __future__ import print_,from django.db import models from django.utils.translation import ugettext_lazy as _ from
`while`,while(1) while(1) { // do something } \end{code} Comment,"while True: try: print(""Enter a number:"") number = int(input())"
`try`,try to find the best solution for your needs. We are a team of professionals with a long experience in the field of,try: from urllib.parse import urlparse except ImportError: from urlparse import urlparse
`if`,if ( !window.atmosphere ) { window.atmosphere = {}; } (function (),if ( ! defined( 'ABSPATH' ) ) { exit; } /** * @package
`for`,for the 2019-2020 school year. The application process for the 2019,for (var i = 0; i < 100000000000000
`torch`,torchbearer 2017-05-18 19:55:25 UTC #,torch.setdefaulttensortype('torch.FloatTensor') def _get_batch_size


If you reach this: congratulations! you've completed everything in this practice session.

If you want to dig deeper, try to implement prompt-tuning (for bonus points!).
You can read more about prompt tuning variants in paper [1](https://arxiv.org/abs/2104.08691) or paper [2](https://arxiv.org/abs/2101.00190). Both versions can be implemented by passing trainable prompts as `model.forward(..., past_key_values=your_prompts)`.



### Read more

* How post-training quantization works: https://arxiv.org/abs/2208.07339
* An overview of running large models: https://huggingface.co/docs/accelerate/package_reference/big_modeling
* A general library for different adapter types: https://adapterhub.ml/


### [extra info] Running other models.

This notebook's code can run with other models of similar size, such as [Falcon-7B](https://huggingface.co/tiiuae/falcon-7b), [OPT-6.7B](https://huggingface.co/facebook/opt-6.7b) or [BLOOM-7.1B](https://huggingface.co/bigscience/bloom-7b1). However, they will require minor code tweaks:
1. change the model name in `AutoModelForCausalLM.from_pretrained()` __and__ `AutoTokenizer`
2. In the prompt tuning code, change `model.model.embed_tokens` to refer to the target model's word embeddings. Simply `print(model)` to navigate to them.
3. Change code to add Lora layers - specifically where you what the transformer block components, since those components now have different names.